## Working with Snapshot camera

Idea: find only the large cluster to reduce detection noise. Then, use the lifespan of customer's cluster to find the most probable vehicle in the snapshot.
Snapshot cam: takes photo every 1:15 second

In [1]:
import sys

sys.path.append("/home/jeans/internship/parking-customer-count")

from countpassenger.Config import conf
from countpassenger import Preprocess
from countpassenger import Cluster
from countpassenger import Utils
from countpassenger import Snapshot

import countpassenger
import pandas as pd
import numpy as np

import os.path as osp
import os

start = 1714284000
stop= 1714288000


pd.set_option("display.precision", 10)
CURRENT_CAM = "mbk-14-14"
SAVE_DIR = os.path.join(conf.RESOURCES_PROCESSED_DIR, f'2024-04-28/count-passengers-{CURRENT_CAM}-vehicle-snapshot.csv')

df_vehicle = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-vehicle-object-20240428.csv")
)
df_cross = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-raw-cross-object-20240428.csv")
)
df_reverse = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-raw-reverse-object-20240428.csv")
)

df_vehicle = Preprocess.df_clean_vehicle(
    df_vehicle=df_vehicle,
)
df_vehicle = Preprocess.filter_camera(df_vehicle, camera_name=CURRENT_CAM + "-vehicle-snapshot")
df_vehicle = Preprocess.sort_df(df_vehicle, conf.VEHICLE_INTEREST_NON_SNAPSHOT)

print(df_vehicle[[*conf.VEHICLE_INTEREST_SNAPSHOT]].to_string())

                          camera       timestamp_precise  timestamp_unix  timestamp_unix_end vehicle_type    xmid    ymid plate_number
2308  mbk-14-14-vehicle-snapshot 2024-04-28 02:26:52.694      1714271212          1714271249    motorbike   907.5   563.0          NaN
2249  mbk-14-14-vehicle-snapshot 2024-04-28 03:04:22.572      1714273462          1714273499        truck  2164.0    58.5       ทห1490
2235  mbk-14-14-vehicle-snapshot 2024-04-28 03:05:02.000      1714273502          1714273505        truck  2167.0    58.5       ทห1490
2236  mbk-14-14-vehicle-snapshot 2024-04-28 03:06:16.105      1714273576          1714273613          bus  2166.5    58.5       ทห1490
2237  mbk-14-14-vehicle-snapshot 2024-04-28 03:07:31.083      1714273651          1714273688          bus  2163.0    58.5       ทห1490
2222  mbk-14-14-vehicle-snapshot 2024-04-28 03:10:01.000      1714273801          1714273802          bus  2166.5    58.5       ทห1490
2223  mbk-14-14-vehicle-snapshot 2024-04-28 03:11:15.10

In [2]:
df_cross = Preprocess.df_clean_customer(df_cross)
df_cross = Preprocess.filter_camera(df_cross, camera_name=CURRENT_CAM)
df_cross = Preprocess.sort_df(df_cross, conf.CUSTOMER_INTEREST_SNAPSHOT)

print(df_cross[[*conf.CUSTOMER_INTEREST_SNAPSHOT,'timestamp_unix']].to_string())

df_reverse = Preprocess.df_clean_customer(df_reverse)
df_reverse = Preprocess.filter_camera(df_reverse, camera_name=CURRENT_CAM)
df_reverse = Preprocess.sort_df(df_reverse, conf.CUSTOMER_INTEREST_SNAPSHOT)

print(df_reverse[[*conf.CUSTOMER_INTEREST_SNAPSHOT, "timestamp_unix"]].to_string())

         camera       timestamp_precise    xmid    ymid  timestamp_unix
2725  mbk-14-14 2024-04-28 02:07:59.195  1989.0   197.5      1714270079
2726  mbk-14-14 2024-04-28 02:08:02.354  1967.5   203.5      1714270082
2719  mbk-14-14 2024-04-28 02:11:47.735   592.0   958.5      1714270307
2720  mbk-14-14 2024-04-28 02:11:47.735  2313.0   158.5      1714270307
2721  mbk-14-14 2024-04-28 02:14:12.454   672.5   949.5      1714270452
2718  mbk-14-14 2024-04-28 02:19:28.528   556.5   907.0      1714270768
2715  mbk-14-14 2024-04-28 02:20:15.761  2198.5   118.0      1714270815
2714  mbk-14-14 2024-04-28 02:20:54.111   734.5   928.0      1714270854
2695  mbk-14-14 2024-04-28 02:31:27.220  1592.0   411.5      1714271487
2696  mbk-14-14 2024-04-28 02:34:19.572  1067.0   731.5      1714271659
2691  mbk-14-14 2024-04-28 02:35:31.236   386.0   930.5      1714271731
2690  mbk-14-14 2024-04-28 02:38:13.951  2191.5   118.0      1714271893
2684  mbk-14-14 2024-04-28 02:39:53.563  1119.0   752.5      171

### Cluster only large vehicle 

In [3]:
params = dict(
    min_cluster_size=2, #have at least 5 people together
    min_samples=2,
    cluster_selection_epsilon=0, 
    max_cluster_size=30,
    alpha=1.0,
    algorithm="brute",
    n_jobs=None,
    cluster_selection_method="eom", # eom with minsample = 3 is kinda good
    store_centers=None,
)

df_cross, cluster_cross = Cluster.perform_cross_clustering(df_cross=df_cross,params=params)
print(Preprocess.sort_df(cluster_cross, conf.CLUSTER_INTEREST)[conf.CLUSTER_INTEREST][(cluster_cross["timestamp_unix_max"] >= start) & (cluster_cross["timestamp_unix_min"] <= stop)].to_string())

/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:58: RuntimeWarning: overflow encountered in exp
  sigmoid = 1 / (1 + np.exp(-k * (x - n)))
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:60: RuntimeWarning: overflow encountered in exp
  exponential_part = np.exp(x - n) + n - 1


           timestamp_min        timestamp_max  count  cluster_id        xmid_mean       ymid_mean
0    2024-04-28 02:31:27  2024-04-28 16:56:45     60          -1  1334.5333333333  577.3250000000
94   2024-04-28 06:05:40  2024-04-28 06:06:06      3          93   688.1666666667  917.3333333333
179  2024-04-28 06:16:03  2024-04-28 06:16:04      2         178   535.5000000000  971.0000000000
180  2024-04-28 06:17:15  2024-04-28 06:17:16      2         179   584.7500000000  859.0000000000
170  2024-04-28 06:18:32  2024-04-28 06:18:32      2         169   599.7500000000  982.5000000000
197  2024-04-28 06:31:11  2024-04-28 06:31:17      2         196   950.2500000000  828.0000000000
181  2024-04-28 06:31:13  2024-04-28 06:31:38      8         180  2123.3750000000  157.1875000000
190  2024-04-28 06:31:14  2024-04-28 06:31:18      2         189  1198.5000000000  682.7500000000
198  2024-04-28 06:31:21  2024-04-28 06:31:28      3         197   651.8333333333  973.8333333333
201  2024-04-28 06:3

/home/jeans/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/cluster/_hdbscan/hdbscan.py:264: UserWarning: The minimum spanning tree contains edge weights with value infinity. Potentially, you are missing too many distances in the initial distance matrix for the given neighborhood size.
  warn(
/tmp/ipykernel_57879/2290256830.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(Preprocess.sort_df(cluster_cross, conf.CLUSTER_INTEREST)[conf.CLUSTER_INTEREST][(cluster_cross["timestamp_unix_max"] >= start) & (cluster_cross["timestamp_unix_min"] <= stop)].to_string())


In [4]:
params = dict(
    min_cluster_size=2,
    min_samples=None,
    cluster_selection_epsilon=0,
    max_cluster_size=30,
    alpha=1.0,
    algorithm="brute",
    cluster_selection_method="eom",
    store_centers=None,
)

df_reverse, cluster_reverse = Cluster.perform_reverse_clustering(df_reverse=df_reverse,params=params)
print(Preprocess.sort_df(cluster_reverse, conf.CLUSTER_INTEREST)[(cluster_reverse["timestamp_unix_max"] >= start) & (cluster_reverse["timestamp_unix_min"] <= stop)][conf.CLUSTER_INTEREST].to_string())

/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:60: RuntimeWarning: overflow encountered in exp
  exponential_part = np.exp(x - n) + n - 1
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:58: RuntimeWarning: overflow encountered in exp
  sigmoid = 1 / (1 + np.exp(-k * (x - n)))


           timestamp_min        timestamp_max  count  cluster_id        xmid_mean       ymid_mean
0    2024-04-28 02:43:34  2024-04-28 15:12:13     24          -1  1400.4583333333  470.8541666667
54   2024-04-28 06:02:37  2024-04-28 06:11:39      5          53  1490.6000000000  390.5000000000
55   2024-04-28 06:16:53  2024-04-28 06:17:48      3          54  1822.1666666667  244.0000000000
95   2024-04-28 06:32:14  2024-04-28 06:34:23      3          94  1768.6666666667  347.5000000000
102  2024-04-28 06:36:32  2024-04-28 06:36:59      2         101  1076.5000000000  647.0000000000
103  2024-04-28 06:38:29  2024-04-28 06:39:24      3         102  1287.3333333333  654.1666666667
33   2024-04-28 07:04:07  2024-04-28 07:06:05      5          32  2062.9000000000  186.4000000000


/home/jeans/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/cluster/_hdbscan/hdbscan.py:264: UserWarning: The minimum spanning tree contains edge weights with value infinity. Potentially, you are missing too many distances in the initial distance matrix for the given neighborhood size.
  warn(
/tmp/ipykernel_57879/3749486777.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(Preprocess.sort_df(cluster_reverse, conf.CLUSTER_INTEREST)[(cluster_reverse["timestamp_unix_max"] >= start) & (cluster_reverse["timestamp_unix_min"] <= stop)][conf.CLUSTER_INTEREST].to_string())


lifespan of customer's cluster to find the most probable vehicle in the snapshot.
Snapshot cam: takes photo every 1:15 second

nearest x,y ? in time windows?

In [5]:
ken = Snapshot.match_cross_cluster_snapshot_to_vehicle(df_vehicle_snapshot=df_vehicle, cluster_cross=cluster_cross,capture_window=70)
johnny = Snapshot.match_reverse_cluster_snapshot_to_vehicle(df_vehicle_snapshot=ken, cluster_reverse=cluster_reverse,capture_window=70)
# johnny[(johnny["timestamp_unix"] >= start) & (johnny["timestamp_unix_end"] <= stop)][conf.VEHICLE_INTEREST_SNAPSHOT_CLUSTERED]
print(johnny.to_string())

      is_plate_valid  travel_distance  vehicle_type_confidence        lifetime  ymin plate_number  xmin  province_confidence similarized_plate_number  plate_number_confidence  xmax                                                                                               image_name purchasing_power province                      camera  travel_displacement            timestamp             car_brand  travel_trajectory  frame_no                        original_video_name  ymax  plate_color_confidence vehicle_type  car_brand_confidence plate_color  plate_image_name       timestamp_precise           true_video_name                                 video_name    xmid    ymid  timestamp_unix  timestamp_unix_end                        cluster_cross_list  cross_count  cluster_reverse_list  reverse_count
2308           False    38.9982720278             0.5153205991   37.5899999142     2          NaN   294                  NaN                      NaN                      NaN  1521      2024-0

Exporting as csv

In [6]:
johnny.to_csv(path_or_buf=SAVE_DIR, index = False)
# johnny.to_csv(path_or_buf=SAVE_DIR, index = False,date_format=str)
